### Project 5: Contagion risk mapper
**Goal**:
Simulate infection spread using symptom onset and contact logs — and flag crew members who had close contact with a symptomatic individual in the days leading up to their symptom onset.

In [2]:
import pandas as pd

In [3]:
contagion_log = pd.read_csv('./data/contagion_log.csv')

In [4]:
contagion_log.head()

,date,crew_member,contact_with,symptoms
0,2025-03-20,Alice Carter,Ben O'Hara,No
1,2025-03-20,Ben O'Hara,Alice Carter,No
2,2025-03-20,Jonas Müller,Nina Patel,No
3,2025-03-20,Nina Patel,Leo Zhang,No
4,2025-03-20,Leo Zhang,Nina Patel,No


In [5]:
symptomatic = contagion_log[contagion_log['symptoms'] == 'Yes']

In [6]:
symptomatic

,date,crew_member,contact_with,symptoms
45,2025-03-27,Nina Patel,Leo Zhang,Yes
47,2025-03-27,Emma López,Alice Carter,Yes


In [35]:
def contact(names):
    for name in names:
        filter_name = contagion_log[contagion_log['crew_member']== name]
        filter_name = filter_name[(filter_name['date'] >= '2025-03-24') & (filter_name['date'] <='2025-03-26')]
        print(f"\n{filter_name}")

In [36]:
contact(['Nina Patel','Emma López'])


          date crew_member  contact_with symptoms
27  2025-03-24  Nina Patel  Alice Carter       No
33  2025-03-25  Nina Patel  Jonas Müller       No
39  2025-03-26  Nina Patel    Emma López       No

          date crew_member  contact_with symptoms
29  2025-03-24  Emma López  Alice Carter       No
35  2025-03-25  Emma López    Ben O'Hara       No
41  2025-03-26  Emma López  Jonas Müller       No
